<a href="https://colab.research.google.com/github/kalz2q/mycolabnotebooks/blob/master/rust01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
%%capture
!apt install rustc;

In [13]:
!rustc --version

rustc 1.43.0


In [14]:
%%writefile main.rs
fn main() {
    println!("Hello, world!");
}

Overwriting main.rs


In [15]:
!rustc main.rs
!./main

Hello, world!


In [16]:
# 実験
%%writefile hoge.rs
fn main() {
    println!("hoge, world!");
}

Overwriting hoge.rs


In [17]:
!rustc hoge.rs; ./hoge

hoge, world!
